# Homework07

Exercises to practice pandas, data analysis and regression

## Goals

- Understand the effects of pre-processing data
- Get familiar with the ML flow: encode -> normalize -> train -> evaluate
- Understand the difference between regression and classification tasks
- Build intuition for different regression models

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor

from data_utils import object_from_json_url
from data_utils import regression_error

### Load Dataset

Let's load up the full [ANSUR](https://www.openlab.psu.edu/ansur2/) dataset that we looked at briefly in [Week 02](https://github.com/DM-GY-9103-2024F-H/WK02).

This is the dataset that has anthropometric information about U.S. Army personnel.

#### WARNING

Like we mentioned in class, this dataset is being used for these exercises due to the level of detail in the dataset and the rigorous process that was used in collecting the data.

This is a very specific dataset and should not be used to draw general conclusions about people, bodies, or anything else that is not related to the distribution of physical features of U.S. Army personnel.

In [ ]:
# Load Dataset
ANSUR_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/main/datasets/json/ansur.json"
ansur_data = object_from_json_url(ANSUR_FILE)

# Look at first 2 records
ansur_data[:2]

#### Nested data

This is that *nested* dataset from Week 02.

# 🤔

Let's load it into a `DataFrame` to see what happens.

In [ ]:
# Read into DataFrame
ansur_df = pd.DataFrame.from_records(ansur_data)
ansur_df.head()


# 😓🙄

That didn't work too well. We ended up with objects in our columns.

Luckily, our `DataFrame` library has a function called [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) that can help.

In [ ]:
# Read into DataFrame
ansur_df = pd.json_normalize(ansur_data)
ansur_df.head()

Much better. `DataFrames` are magic.

#### Data Exploration

Before we start creating models, let's do a little bit of data analysis and get a feeling for the shapes, distributions and relationships of our data.

1. Print `min`, `max` and `average` values for all of the features.
2. Print `covariance` tables for `age`, `ear.length` and `head.circumference`.
3. Plot `age`, `ear.length` and `head.circumference` versus the $1$ *feature* that is most correlated to each of them.

Don't forget to *encode* and *normalize* the data.

In [ ]:
# Work on Data Exploration here

### Encode non-numerical features
# gender is non-numerical and has two possible values: F and M
# since the values don't have an order, we can use OneHotEncoder, which turns each value into a distinct feature and uses 0 and 1 to mark whether a record has a value or not

from sklearn.preprocessing import OneHotEncoder
ansur_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
gender_encoded = ansur_encoder.fit_transform(ansur_df[['gender']])
# getting the new column names
# since ansur_encoder.categories creates a list of arrays (one array per feature), we access the first element of the list
new_col_names = ansur_encoder.categories_[0]
ansur_encoded_gender_df = pd.DataFrame(gender_encoded, columns=new_col_names, index=ansur_df.index)

# concatenating the new columns to the original dataframe and dropping the original gender column
ansur_encoded_df = pd.concat([ansur_df, ansur_encoded_gender_df], axis=1)
ansur_encoded_df = ansur_encoded_df.drop(columns=['gender'])

# checking the new dataframe and confirming is has been encoded correctly
# print(ansur_encoded_df.head())


## 1. Print min, max, avg
max_values = ansur_encoded_df.max()
min_values = ansur_encoded_df.min()
avg_values = ansur_encoded_df.mean()
# print("max values are:\n", max_values)
# print("min values are:\n", min_values)
# print("avg values are:\n", avg_values)

### Normalize all data
# using StandardScaler, to represent each data point in terms of the aggregate characteristics for that collection of points
ansur_scaler = StandardScaler().set_output(transform="pandas")
ansur_scaled_df = scaler.fit_transform(ansur_encoded_df)

## 2. Print Covariances
display(ansur_scaled_df.cov())

## 3. Plot features most correlated to age, ear length and head circumference
# features most correlated to age are ear.length, weight, ear.breadth, and hand.breadth
# ear.length has strong correlation with different features: weight, hand.breadth, ear.breadth, stature, etc
# the same applies to head circumference, with head.height, weight, foot.length	, and foot.breadth being the most correlated features

# plot age as a function of ear.length, weight, ear.breadth, and hand.breadth
for feat in ["ear.length", "weight", "ear.breadth", "hand.breadth"]:
  plt.plot(ansur_scaled_df[feat], ansur_scaled_df["age"], marker="o", color="r", linestyle="", alpha=0.4)
  plt.xlabel(feat)
  plt.ylabel("age")
  plt.show()

# plot ear.length as a function of weight, hand.breadth, ear.breadth, stature
for feat in ["weight", "hand.breadth", "ear.breadth", "stature"]:
  plt.plot(ansur_scaled_df[feat], ansur_scaled_df["ear.length"], marker="o", color="b", linestyle="", alpha=0.4)
  plt.xlabel(feat)
  plt.ylabel("ear.length")
  plt.show()
# these plots show a stronger, more discernible positive correlation between ear.length and the other features

# plot head.circumference as a function of head.height, weight, foot.length, and foot.breadth
for feat in ["head.height", "weight", "foot.length", "foot.breadth"]:
  plt.plot(ansur_scaled_df[feat], ansur_scaled_df["head.circumference"], marker="o", color="g", linestyle="", alpha=0.4)
  plt.xlabel(feat)
  plt.ylabel("head.circumference")
  plt.show()
# these plots also show a strong positive correlation between head.circumference and the other features



### Interpretation

<span style="color:hotpink;">
Does anything stand out about these graphs? Or the correlations?<br>
Are correlations symmetric? Does the feature most correlated to ear length also have ear length as its most correlated pair?
</span>

The plots for age should show a positive correlation between age and the other features, but the correlation is subtle so it's almost visually undiscernible in all cases except for ear.length, which shows a more pronounced positive correlation.

In contrast, the plots for ear.length and head.circumference show a clearer, strong correlation between the variables and the respective correlated features.
Overall, this suggests that age is not the stronger indicator when it comes to predicting the vaues for other features.

Correlations are symmetric, meaning that the correlation between variable X and variable Y is identical to the correlation between variable Y and variable X. But this does not mean that the feature most correlated with a variable also has to have that variable as its most correlated pais. For example, the feature with which ear.length has the strongest correlation is weight. But weight shows more correlation with several other values than with ear.length, such as hand.breadth, foot.breadth, foot.length, and stature.

### Regression

Now, we want to create a regression model to predict `head.circumference` from the data.

From our [Week 07](https://github.com/PSAM-5020-2025F-A/WK07) notebook, we can create a regression model by following these steps:

1. Load dataset (done! 🎉)
2. Encode label features as numbers (done! ⚡️)
3. Normalize the data (done! 🍾)
4. Separate the outcome variable and the input features
5. Create a regression model using all features
6. Run model on training data and measure error
7. Plot predictions and interpret results
8. Run model on test data, measure error, plot predictions, interpret results

In [ ]:
# Work on Regression Model here

## Separate outcome variable and input features
head_circumference = ansur_scaled_df[["head.circumference"]]
# using only 3 features that are most correlated to head.circumference, so that we can then plot and visualize the results more easily
features = ansur_scaled_df[["head.height", "weight", "foot.length", "foot.breadth"]]

## Create a regression model
head_circumference_model = LinearRegression().fit(features, head_circumference)

## Measure error on training data
predicted_head_circumference = head_circumference_model.predict(features)
error = regression_error(head_circumference, predicted_head_circumference)
print("error on training data:", error)

## Plot predictions and interpret results

plt.plot(head_circumference, marker='o', linestyle='', alpha=0.3)
plt.plot(predicted_head_circumference, color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("Head circumference (actual (blue) vs predicted (red))")
plt.legend()
plt.show()

# another way to visualize how far off the model is from the actual data is to look at sorted lists of head circumferences
# this lets us see if there are ranges where the model is wrong/right more often
plt.plot(sorted(head_circumference.values), marker='o', linestyle='', alpha=0.3)
plt.plot(sorted(predicted_head_circumference), color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("had circumference (actual (blue) vs predicted (red))")
plt.show()

# and since there are more than 3 variables to map, we can just plot head circumference as a function of a few individual features
# plot head.circumference vs head.height, weight, foot.length, foot.breadth
for feat in ["head.height", "weight", "foot.length", "foot.breadth"]:
  x = ansur_scaled_df[feat]
  head_circumference_original = ansur_scaled_df["head.circumference"]
  head_circumference_predicted = predicted_head_circumference
  # Plot the original and predicted head circumferences
  plt.plot(x, head_circumference_original, marker='o', linestyle='', alpha=0.3)
  plt.plot(x, head_circumference_predicted, color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
  plt.xlabel(feat)
  plt.ylabel("head circumference")
  plt.show()

# we can see that the predicted trend is correct, but there is a large error margin in the predictions
# the middle ranges of the head circumference values seem to have less error than the extreme ends (seen in the sorted plot)


In [28]:
## Load Test Data
ANSUR_TEST_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/main/datasets/json/ansur-test.json"

ansur_test_data = object_from_json_url(ANSUR_TEST_FILE)
ansur_test_df = pd.json_normalize(ansur_test_data)

ansur_test_encoded_df = ansur_test_df.copy()

g_vals = ansur_encoder.transform(ansur_test_df[["gender"]].values)
ansur_test_encoded_df[["gender"]] = g_vals

ansur_test_scaled_df = ansur_scaler.transform(ansur_test_encoded_df)

/usr/local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


ValueError: Columns must be same length as key

In [ ]:
## Run model on test data

## Measure error on test data

## Plot predictions and interpret results

### Interpretation

<span style="color:hotpink;">
How well does your model perform?<br>
How could you improve it?<br>
Are there ranges of circumferences that don't get predicted well?
</span>

<span style="color:hotpink;">EDIT THIS CELL WITH ANSWER</span>